In [1]:
import numpy
import random
seed_bytes = b'\xc2\x91\xc2\x91\xc2\x88\xc2\x9a\xc2\xa8\xc2\x98a\xc2\x97e\xc2\xa9\xc2\x8e\xc2\x90\xc2\x82s\xc2\x9c\xc2\x8a\xc2\x8d\xc2\x97m\xc2\x99\xc2\x8d\xc2\x97\xc2\xa1\xc2\xa7'
random.seed(seed_bytes)
numpy.random.seed(int.from_bytes(seed_bytes[:4], "big"))

In [2]:
TARGET_DATASET_VIDEO_ROOT = "/mnt/c/Skripsi/UCF-101-h264"
TARGET_DATASET_GEN = "/mnt/c/Skripsi/UCF-101-pregen/"

In [3]:
from importables.preprocessing_managers import DatasetDictionary 
dataset_dicitonary = DatasetDictionary(TARGET_DATASET_VIDEO_ROOT+".csv", DatasetDictionary.Mappings.UCF101_ACTION_RECOGNITION_DATASET)

In [4]:
dataset_dicitonary_pd = dataset_dicitonary.as_DataFrame()
dataset_dicitonary_pd.head()

,ActionID,ClipID,GroupID,SourceFilePath,GeneratedFileName,FrameCount
0,1,1,1,/mnt/c/Skripsi/UCF-101-h264/ApplyEyeMakeup/v_A...,A001C001G001.h5,165
1,1,2,1,/mnt/c/Skripsi/UCF-101-h264/ApplyEyeMakeup/v_A...,A001C002G001.h5,124
2,1,3,1,/mnt/c/Skripsi/UCF-101-h264/ApplyEyeMakeup/v_A...,A001C003G001.h5,260
3,1,4,1,/mnt/c/Skripsi/UCF-101-h264/ApplyEyeMakeup/v_A...,A001C004G001.h5,236
4,1,5,1,/mnt/c/Skripsi/UCF-101-h264/ApplyEyeMakeup/v_A...,A001C005G001.h5,297


In [5]:
filter_ds = dataset_dicitonary_pd
ds_filtered_tuple_np = filter_ds[dataset_dicitonary_pd.columns[-3:-1]].to_numpy()
ds_filtered_tuple = ds_filtered_tuple_np.tolist()

In [6]:
import os
print("Generating execution file...")
generate_python_runfile = \
f"""
from importables.preprocessing_managers import PreprocessingManagers, DatasetDictionary
from importables.constants import PREPROCESSING_ARGUMENTS_MASK_GENERATOR, PREPROCESSING_ARGUMENTS_MOTION_VECTOR_PROCESSOR, PREPROCESSING_ARGUMENTS_OPTICAL_FLOW_GENERATOR
from tqdm.auto import tqdm
import os
import time
import cv2
import traceback
FILE_TUPLE_LIST = {ds_filtered_tuple}
FOLDER_DEST = "{TARGET_DATASET_GEN}"
log_file = open(FOLDER_DEST+"../logfile.txt","a")
pregenerator = PreprocessingManagers.Pregenerator(
    target_folder=FOLDER_DEST,
    motion_vector_processor_kwargs=PREPROCESSING_ARGUMENTS_MOTION_VECTOR_PROCESSOR,
    mask_generator_kwargs=PREPROCESSING_ARGUMENTS_MASK_GENERATOR,
    optical_flow_generator_kwargs=PREPROCESSING_ARGUMENTS_OPTICAL_FLOW_GENERATOR,
    directory_mapping=DatasetDictionary.Mappings.UCF101_ACTION_RECOGNITION_DATASET
) 
pregenerator.init_models(FILE_TUPLE_LIST[0][0])
last_index = int(input("Start from ?(Index): "))
print("Sleeping for 10 seconds")
time.sleep(10)
print("Running...")
current_idx = 0
retry_count = 0 
MAX_RETRY = 3
for current_path, current_name in tqdm(FILE_TUPLE_LIST):
    if current_idx < last_index:
        print("Skipping...")
        current_idx += 1
        continue
    cap = cv2.VideoCapture(current_path)
    current_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    while True:
        print({r'f"Current Index = {current_idx}"'})
        try:
            current_success = pregenerator.pregenerate_preprocessing(current_path, current_name, current_length)
            assert current_success
        except KeyboardInterrupt:
            exit()
        except:
            traceback.print_exc()
            if retry_count < MAX_RETRY:
                print({r'f"Retrying {current_name}..."'})
                retry_count += 1
                continue
            else:
                print({r'f"Skipping {current_name} for failing more than 10 times..."'})
                current_idx += 1 
                log_file.write("\\n\\n"+"Current Index: " + str(current_idx) +"\\n")
                log_file.write("\\n"+current_name+"\\n\\n")
                log_file.write(traceback.format_exc())
                retry_count = 0
                break
        else:
            current_idx += 1 
            retry_count = 0
            break
    os.system('clear')
""" 
f = open("./preprocess_data_execute.py", "w")
f.write(generate_python_runfile)
f.close()
print("Execution file generated")
print(f"Run execution file independently with this command\n\ncd '{os.path.abspath('./')}';python '{os.path.abspath('./preprocess_data_execute.py')}'\n\nfor maximum performance")
print("This notebook will terminate")

Generating execution file...
Execution file generated
Run execution file independently with this command

cd '/mnt/d/Desktop/kuliah mbut/SKRIPSI BOIS/Codes/HARMV';python '/mnt/d/Desktop/kuliah mbut/SKRIPSI BOIS/Codes/HARMV/preprocess_data_execute.py'

for maximum performance
This notebook will terminate


In [ ]:
os._exit(00)

: 

: 